In [1]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import scipy as sp
from scipy.integrate import odeint
import networkx as nx

from scipy import stats

import pybel as pb
import json
import time
import csv

import torch
import pyro

from bel2scm import graph_node as cn
from bel2scm import gen_test_data as td
from bel2scm import causal_graph as cg

pyro.set_rng_seed(101)
torch.set_default_tensor_type(torch.DoubleTensor)

In [2]:
tot_data = td.data_gen(10000)

# Unit Tests

In [3]:
# Unit Test in icream graph

str_list = ['temp =| cloudy','cloudy => rainy','temp => icream','rainy =| icream']
type_dict = {}
type_dict['temp'] = 'continuous'
type_dict['cloudy'] = 'continuous'
type_dict['rainy'] = 'continuous'
type_dict['icream'] = 'continuous'
graph_test = cg.str_graph(str_list=str_list,graph_type='SCM',type_dict=type_dict)
graph_test.add_confound([['icream','cloudy']])

print(graph_test.d_sep(['icream'],['cloudy'],['rainy','temp']))
#Ans: False
print()
print(graph_test.id_alg(['icream'],['cloudy']))
#Ans: P_{cloudy}(icream) = sum_{rainy} P(rainy|cloudy) sum_{cloudy}′P(icream|cloudy',rainy)P(cloudy')
print()

print(graph_test.idc_alg(['icream'],['cloudy'],['temp']))
#Ans: P_{cloudy}(icream|temp) = [sum_{rainy} P(rainy|cloudy,temp) sum_{cloudy'} P(icream|cloudy',rainy,temp)P(cloudy'|temp)]  
# /[sum_{rainy,icream} P(rainy|cloudy,temp) sum_{cloudy'}P(icream|cloudy',rainy,temp)P(cloudy'|temp)]

The causal graph is a acyclic
False

[sum_{temp,rainy} [sum_{} [sum_{cloudy,rainy,icream} P(temp,cloudy,rainy,icream)]][sum_{}P(rainy|cloudy)][sum_{} [sum_{cloudy} P(cloudy|temp)P(icream|cloudy,rainy,temp)]]]

[[sum_{rainy} [sum_{}P(rainy|cloudy)][sum_{} [sum_{cloudy} P(cloudy|temp)P(icream|cloudy,rainy,temp)]]]]/[ sum_{icream}[sum_{rainy} [sum_{}P(rainy|cloudy)][sum_{} [sum_{cloudy} P(cloudy|temp)P(icream|cloudy,rainy,temp)]]]]


In [4]:
# Unit Test in XYZ Graph

str_list = ['X =| W','D => Z','W => Y','Z =| Y','V =| Z']
type_dict = {}
type_dict['X'] = 'continuous'
type_dict['W'] = 'continuous'
type_dict['D'] = 'continuous'
type_dict['Z'] = 'continuous'
type_dict['Y'] = 'continuous'
type_dict['V'] = 'continuous'

graph_test = cg.str_graph(str_list=str_list,graph_type='SCM',type_dict=type_dict)
graph_test.add_confound([['X','Y']])

print(graph_test.d_sep(['D'], ['Z'],['Y']))
# Ans: True
print()
print(graph_test.id_alg(['Y'],['X']))
# Ans: P_X(Y) = sum_W P(W|X) sum_X' P(Y|X',W)P(X')
print()
print(graph_test.idc_alg(['Y'],['X'],['W']))
# Ans: P_X(Y|W) = [sum_{D,V,Z} P(Z|D,V) P(V) P(D) sum_X' P(Y|D,V,W,X′,Z) P(X'|D,V,Z)]
#/[sum_{D,V,Z,Y} P(Z|D,V) P(V) P(D) sum_x' P(Y|D,V,W,X′,Z) P(X'|D,V,Z)]
print()
print(graph_test.idc_alg(['Y'],['X'],['V']))
# Ans: P_X(Y|V) = [sum_{W,D,Z} P(W|X)P(D) P(Z|D,V)[sum_{X} P(X')P(Y|X',V,W,D,Z)]]
#/[sum_{W,D,Z,Y} P(W|X)P(D) P(Z|D,V)[sum_{X'} P(X)P(Y|X',V,W,D,Z)]]

# Note that the code doesn't automatically put in the ' for summation/dummy variables - you have to
# infer this from the nested brackets; summation only applies to items inside the summation sign

The causal graph is a acyclic
False

[sum_{W,D,Z,V} [sum_{}P(W|X)][sum_{} [sum_{X,W,Z,Y,V} P(X,W,D,Z,Y,V)]][sum_{}P(Z|D,V)][sum_{} [sum_{X} P(Y|X,V,W,D,Z)P(X)]][sum_{} [sum_{X,W,D,Z,Y} P(X,W,D,Z,Y,V)]]]

[[sum_{D,Z,V} [sum_{} [sum_{X,W,Z,Y,V} P(X,W,D,Z,Y,V)]][sum_{}P(Z|D,V)][sum_{} [sum_{X} P(Y|X,V,W,D,Z)P(X)]][sum_{} [sum_{X,W,D,Z,Y} P(X,W,D,Z,Y,V)]]]]/[ sum_{Y}[sum_{D,Z,V} [sum_{} [sum_{X,W,Z,Y,V} P(X,W,D,Z,Y,V)]][sum_{}P(Z|D,V)][sum_{} [sum_{X} P(Y|X,V,W,D,Z)P(X)]][sum_{} [sum_{X,W,D,Z,Y} P(X,W,D,Z,Y,V)]]]]

[[sum_{W,D,Z} [sum_{}P(W|X)][sum_{} [sum_{X,W,Z,Y,V} P(X,W,D,Z,Y,V)]][sum_{}P(Z|D,V)][sum_{} [sum_{X} P(Y|X,V,W,D,Z)P(X)]]]]/[ sum_{Y}[sum_{W,D,Z} [sum_{}P(W|X)][sum_{} [sum_{X,W,Z,Y,V} P(X,W,D,Z,Y,V)]][sum_{}P(Z|D,V)][sum_{} [sum_{X} P(Y|X,V,W,D,Z)P(X)]]]]


In [5]:
# Unit Test in XYZ Graph from Shpitser thesis and paper

str_list = ['X =| W','D => Z','W => Y','Z =| Y']
type_dict = {}
type_dict['X'] = 'continuous'
type_dict['W'] = 'continuous'
type_dict['D'] = 'continuous'
type_dict['Z'] = 'continuous'
type_dict['Y'] = 'continuous'

graph_test = cg.str_graph(str_list=str_list,graph_type='SCM',type_dict=type_dict)
graph_test.add_confound([['X','Y']])


do_temp = [['Y',['X*','Z*','D']],['Z',['D']]]
obs_temp = ['X','D']

do_temp2 = [['Y',['X*','Z*']]]
obs_temp2 = ['X']

do_gam = [['Y',['X*']]]
do_del = [['Z',['D']]]
obs_del = ['X','D']


do_temp3 = [['Y', ['X*', 'Z*']], ['W', ['X*','Z*']]]
obs_temp3 = ['Z', 'X']

def plot_graph(dag, confounders, filename, layout='dot'):
    labels = dict([(node,node) for node in dag.nodes])
    if layout == 'dot':
        pos = nx.drawing.nx_pydot.graphviz_layout(dag, prog="dot")
    else:
        pos = nx.kamada_kawai_layout(dag)
    fig = plt.figure()
    nx.draw(dag, pos, labels=labels, width=4, font_size=8)
    nx.draw(confounders, pos, labels=labels, font_size=8)
    fig.savefig(filename)
    plt.show()

def cf2latex(cf_variables):
    latex = []
    for cf, do in cf_variables:
        latex.append('%s_{%s}' % (cf, ','.join(do)))
    return ','.join(latex)

for cf, do in do_temp:
    print(cf)
    print(do)

The causal graph is a acyclic
Y
['X*', 'Z*', 'D']
Z
['D']


In [35]:
gamma_cg

['Y_{X*,Z*}', 'X', 'D', 'Z']

In [6]:
    
from IPython.display import Latex
 
display(Latex('$$P(%s)$$' % cf2latex(do_temp)))
g_pw,c_pw = graph_test.make_pw_graph(do_temp)
plot_graph(g_pw, c_pw, 'parallel_worlds.png')


dag_cg, confounder_cg, gamma_cg  = graph_test.make_cf_graph( do_temp, obs_temp)  
display(Latex("$$P(%s,%s) = P(%s)$$" % (cf2latex(do_temp), ','.join(obs_temp), ','.join(gamma_cg))))
plot_graph(dag_cg, confounder_cg, 'cg1.png')
print(graph_test.id_star_alg(do_temp,obs_temp))
print( "Ans: \nsum_{W}P(Y_{Z*,W},X)P(D)P(Z_{D})P(W_{X*})")
print()
dag_cg2, confounder_cg2, gamma_cg2  = graph_test.make_cf_graph( do_temp2, obs_temp2)  
display(Latex("$$P(%s,%s) = P(%s)$$" % (cf2latex(do_temp2), ','.join(obs_temp2), ','.join(gamma_cg2))))
plot_graph(dag_cg2, confounder_cg2, 'cg2.png')
print(graph_test.id_star_alg(do_temp2,obs_temp2))
print( "Ans: \nsum_{W}P(Y_{Z*,W},X)P(W_{X*})")
print()

dag_gam, confounder_gam, gamma_gam  = graph_test.make_cf_graph( do_del + do_gam, obs_del)  
display(Latex("$$P(%s|%s) = P(%s)$$" % (cf2latex(do_gam + do_del), ','.join(obs_del), ','.join(gamma_gam))))
plot_graph(dag_gam, confounder_gam, 'cgc.png')
print(graph_test.idc_star_alg(do_gam,do_del,[],obs_del))
print("Ans: \n[sum_{W}P(Y_{Z,W},X)P(W_{X*})]/[sum_{Y}[sum_{W}P(Y_{Z,W},X)P(W_{X*})]]")
print()
dag3, confounders3, gamma3 = graph_test.make_cf_graph( do_temp3, obs_temp3)
display(Latex("$$P(%s, %s)= P(%s)$$" % (cf2latex(do_temp3), ','.join(obs_temp3), ','.join(gamma3 ))))
plot_graph(dag3, confounders3, 'cg3.png')
print(graph_test.id_star_alg(do_temp3,obs_temp3))
print("Ans: \nsum_{D}P(Y_{Z*,W},X)P(D)P(Z_{D})P(W_{X*})")

sum_{W}P(Y_{Z*,W},X)P(D)P(Z_{D})P(W_{X*})
Ans: 
sum_{W}P(Y_{Z*,W},X)P(D)P(Z_{D})P(W_{X*})



sum_{W}P(Y_{Z*,W},X)P(W_{X*})
Ans: 
sum_{W}P(Y_{Z*,W},X)P(W_{X*})



[sum_{W}P(Y_{Z,W},X)P(W_{X*})]/[sum_{Y}[sum_{W}P(Y_{Z,W},X)P(W_{X*})]]
Ans: 
[sum_{W}P(Y_{Z,W},X)P(W_{X*})]/[sum_{Y}[sum_{W}P(Y_{Z,W},X)P(W_{X*})]]



sum_{D}P(Y_{Z*,W},X)P(D)P(Z_{D})P(W_{X*})
Ans: 
sum_{D}P(Y_{Z*,W},X)P(D)P(Z_{D})P(W_{X*})


Parallel Worlds Graph: <img src="shpitser_pw_graph.png">
Counterfactual Graph<img src="shpitser_cf_graph.png">

$$P'=\sum_wP_{z,w}(y,x')P_x(w) = \sum_wP(y_{z,w}, x_{z,w}')P(w_x) = \sum_wP(y_{z,w}, x')P(w_x)$$